# Decoders for quantum error correction

# Belief propagation + ordered statistics decoding (BP+OSD)

The belief propagation + ordered statistics decoder (BP+OSD) is a modified version of BP that works well for quantum LDPC codes. An advantage of the BP+OSD decoder is that it always returns a decoding that satisfies the syndrome equation.

The `LDPC` implementation of BP+OSD inherits from the `ldpc.BpDecoder` class. The setup is therefore very similar, with only two extra parameters to specify the `osd_method` and the `osd_order`. An example is below:

In [ ]:
import numpy as np
import ldpc.codes
from ldpc import BpOsdDecoder

H = ldpc.codes.hamming_code(5)

## The 
bp_osd = BpOsdDecoder(
            H,
            error_rate = 0.1,
            bp_method = 'product_sum',
            max_iter = 7,
            schedule = 'serial',
            osd_method = 'osd_cs',
            osd_order = 2
        )

syndrome = np.random.randint(size=H.shape[0], low=0, high=2).astype(np.uint8)

print(f"Syndrome: {syndrome}")
decoding = bp_osd.decode(syndrome)
print(f"Decoding: {decoding}")
decoding_syndrome = H@decoding % 2
print(f"Decoding syndrome: {decoding_syndrome}")


# Belief-find

The belief propagation + union-find decoder (belief-find) is an alternative to BP+OSD for decoding quantum LDPC codes. Belief-find first attempts to solve the decoding using belief propagation. If this fails, a union-find decoder is run as a post-processor using the output of BP to guide cluster growth. The `LDPC` package implements two versions of belief-find:

1) **Peeling belief-find**. This approach uses a peeling decoder to solve the local decoding problem on each code. Peeling uinon-find is limited to decoding codes that have point like syndromes. e.g, the surface and toric codes. The peeling version of union-find decoder was first proposed by Delfosse & Nickerson in (https://arxiv.org/abs/1709.06218). Peeling belief-find was first proposed and implemented by Oscar Higgott (https://arxiv.org/abs/2203.04948).

2) **Cluster-inversion belief-find**. This approach solves the local decoding problem on each cluster using matrix inverison. Cluster-inversion union find can be applied to any quantum LDPC code. Matrix inversion find was first proposed by Delfosse et al. (https://arxiv.org/abs/2103.08049), and later improved by Berent et al. (https://arxiv.org/abs/2209.01180). As far as I am aware, the `LDPCv2` has the first implementation of cluster-inversion belief-find.

The `LDPC` implementation of belief-find inherits from the `ldpc.BpDecoder` class. Examples of belief-find in use are given below:

In [ ]:
import numpy as np
import ldpc.codes
from ldpc import BeliefFindDecoder

H = ldpc.codes.hamming_code(5)

## The 
bf = BeliefFindDecoder(
            H,
            error_rate = 0.1,
            bp_method = 'product_sum',
            max_iter = 7,
            schedule = 'serial',
            matrix_solve = True, # If set to True, union-find clusters are solved by matrix inversion
            bits_per_step = 1 ## this is the number of bits by which clusters are expanded in each growth step 
        )

syndrome = np.random.randint(size=H.shape[0], low=0, high=2).astype(np.uint8)

print(f"Syndrome: {syndrome}")
decoding = bf.decode(syndrome)
print(f"Decoding: {decoding}")
decoding_syndrome = H@decoding % 2
print(f"Decoding syndrome: {decoding_syndrome}")

The peeling version of belief-find can be activated by setting `matrix_solve=False`. This method only works for parity check matrix that yield point-like syndromes. An example of using peeling belief-find on the repetition code is show below:

In [10]:
import numpy as np
import ldpc.codes
from ldpc import BeliefFindDecoder

H = ldpc.codes.ring_code(15)

## The 
bf = BeliefFindDecoder(
            H,
            error_rate = 0.1,
            bp_method = 'product_sum',
            max_iter = 7,
            schedule = 'serial',
            matrix_solve = False, # If matrix_solve is set to False, union-find clusters are solved using a peeling decoder
            bits_per_step = 1 ## this is the number of bits by which clusters are expanded in each growth step 
        )



In [13]:
error = np.random.randint(size=H.shape[0], low=0, high=2).astype(np.uint8)
syndrome = H@error % 2

decoding = bf.decode(syndrome)
print(f"Decoding: {decoding}")
decoding_syndrome = H@decoding % 2
print(f"Decoding syndrome: {decoding_syndrome}")

Decoding: [1 1 1 1 1 1 0 0 0 0 0 0 1 0 0]
Decoding syndrome: [0 0 0 0 0 1 0 0 0 0 0 1 1 0 1]
